In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.chrome.options import Options ### CONFIGURADO CON EDGE
import pandas as pd
from datetime import datetime, timedelta
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
#import cv2
#import pytesseract
import base64
import numpy as np
import os
import random
import string

In [95]:
def unicos_columna(df, numero):
    print("DATOS COLUMNA ", df.columns[numero])
    print(df[df.columns[numero]].value_counts()) 

In [96]:
options = Options()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
options.add_argument("--disable-notifications")

def opendriver(options):
    driver = webdriver.Chrome(options=options)
    time.sleep(2)
    driver.maximize_window()
    print("Abriendo WebDriver")
    time.sleep(2)
    return driver

driver = opendriver(options)

Abriendo WebDriver


In [97]:
busqueda = "La Serena"
def apertura_marketplace(driver, busqueda):
    airbnburl = "https://www.airbnb.cl/"
    driver.get(airbnburl)
    time.sleep(3)
    print("Cargando Airbnb y realizando búsqueda de '", busqueda, "'")
    buscador = driver.find_element(By.XPATH,'//div[contains(text(), "En cualquier lugar del mundo")]/..')
    buscador.click()
    time.sleep(1)
    buscador = driver.find_element(By.XPATH, '//input[@id="bigsearch-query-location-input"]')
    #buscador.clear()
    #buscador.send_keys(Keys.CONTROL, "a")
    #buscador.send_keys(Keys.DELETE)
    buscador.send_keys(busqueda)
    button = driver.find_element(By.XPATH,'//button[@data-testid="structured-search-input-search-button"]')
    button.click()
    time.sleep(3)
apertura_marketplace(driver, busqueda)

#<input class="ivycze5 dir dir-ltr" aria-autocomplete="none" autocomplete="off" autocorrect="off" spellcheck="false" id="bigsearch-query-location-input" name="query" aria-describedby="bigsearch-query-location-description" placeholder="Explora destinos" data-testid="structured-search-input-field-query" value="">

Cargando Airbnb y realizando búsqueda de ' La Serena '


In [98]:
def aplicar_filtros(driver):
    
    button = driver.find_element(By.XPATH, '//button[@style="--filter-button_border:1px solid var(--j-qkgmf);"]')
    button.click()
    time.sleep(1)
    
    divs = driver.find_element(By.XPATH, '(//*[@id="site-content"])[2]')
    
    options = divs.find_elements(By.XPATH, './div[3]//section/div[2]/div')
    rooms = options[0].find_elements(By.XPATH, ".//button")
    beds = options[1].find_elements(By.XPATH, ".//button")
    baths = options[2].find_elements(By.XPATH, ".//button")

    filtered_rooms = []
    filtered_beds = []
    filtered_baths = []

    def filter_noptions(lista1, lista2):
        for element in lista1:
            if not any(val in element.text for val in ["Cualquiera", "5", "6", "7", "8+", "4"]):
                lista2.append(element)
        #for filtered in lista2:
        #    print(filtered.text)
    
    #closebutton = driver.find_element(By.XPATH, '//button[@aria-label="Cerrar"]')
    
    filter_noptions(rooms, filtered_rooms)
    filter_noptions(beds, filtered_beds)
    filter_noptions(baths, filtered_baths)

    random.choice(filtered_rooms).click()
    random.choice(filtered_beds).click()
    random.choice(filtered_baths).click()

    filterbutton = driver.find_element(By.XPATH, '//a[contains(text(), "Mostrar")]')
    time.sleep(1)
    print(''.join(filter(str.isdigit, filterbutton.text)), " elementos encontrados")

    def tipo_filtro(divs):
        options = divs.find_elements(By.XPATH, './div[4]//button')
        print(len(options))
        buttontipo = random.choice(options)
        buttontipo.click()
        return buttontipo.text

    if int(''.join(filter(str.isdigit, filterbutton.text))) > 270:
        print("Segundo Filtro")
        tipo_vivienda = tipo_filtro(divs)
        print("TIPO SELECCIONADO :  <<< ",tipo_vivienda, " >>>")
        time.sleep(1)
        filterbutton = driver.find_element(By.XPATH, '//a[contains(text(), "Mostrar")]')
        print("Elemtos reducidos a ",''.join(filter(str.isdigit, filterbutton.text)), " objetos")    
    
    def servicio_filtro(divs):
        options = divs.find_elements(By.XPATH, './div[5]//span[@data-checkbox="true"]')
        print(len(options))
        random.choice(options).click()
        time.sleep(1)
        filterbutton = driver.find_element(By.XPATH, '//a[contains(text(), "Mostrar")]')
        if int(''.join(filter(str.isdigit, filterbutton.text))) > 270:
            print("Filtro Extra")
            servicio_filtro(divs)
    
    if int(''.join(filter(str.isdigit, filterbutton.text))) > 270:
        print("Tercer Filtro")
        button = divs.find_element(By.XPATH, './div[5]//span[contains(text(), "Muestra")]/..')
        button.click()
        time.sleep(2)
        servicio_filtro(divs)
        time.sleep(1)
        filterbutton = driver.find_element(By.XPATH, '//a[contains(text(), "Mostrar")]')
        time.sleep(1)
        print("Elemtos reducidos a ",''.join(filter(str.isdigit, filterbutton.text)), " objetos")

    filterbutton.click()

aplicar_filtros(driver)

1000  elementos encontrados
Segundo Filtro
4
TIPO SELECCIONADO :  <<<  Hotel  >>>
Elemtos reducidos a  115  objetos


In [99]:
def scrap(driver, link, df):
    driver.get(link)
    time.sleep(3)
    googlemap = driver.find_element(By.XPATH, '//h2[contains(text(), "A dónde irás")]')
    driver.execute_script("arguments[0].scrollIntoView({behavior: 'instant', block: 'center'});", googlemap)
    time.sleep(4)
            
    Link = {"Link":link}
    print(Link)
    Comuna = {"Comuna": driver.find_element(By.XPATH, '//*[@id="site-content"]/div/div[1]/div[1]/div[1]/div/div/div/div/section/div[2]/div[1]/span/button/span').text}
    Camas = {"Camas": driver.find_element(By.XPATH, '//*[@id="site-content"]/div/div[1]/div[3]/div/div[1]/div/div[1]/div/div/section/.//span[contains(text(), "cama")]').text}
    Baños = {"Baños": driver.find_element(By.XPATH, '//*[@id="site-content"]/div/div[1]/div[3]/div/div[1]/div/div[1]/div/div/section/.//span[contains(text(), "baño")]').text}
    Capacidad = {"Capacidad":driver.find_element(By.XPATH, '//*[@id="site-content"]/div/div[1]/div[3]/div/div[1]/div/div[1]/div/div/section/.//span[contains(text(), "huésped")]').text}
    Habitaciones = {"Habitaciones":driver.find_element(By.XPATH, '//*[@id="site-content"]/div/div[1]/div[3]/div/div[1]/div/div[1]/div/div/section/.//span[contains(text(), "habitacion")]').text}
    try: 
        if driver.find_element(By.XPATH, '//*[@id="site-content"]/div/div[1]/div[3]/div/div[2]/div/div/div[1]/div/div/div/div/div/div/div[1]/div[1]/div[1]/div/span/div/span[2]').text == "noche":
            Precio = {"Precio":driver.find_element(By.XPATH, '//*[@id="site-content"]/div/div[1]/div[3]/div/div[2]/div/div/div[1]/div/div/div/div/div/div/div[1]/div[1]/div[1]/div/span/div/span[1]').text}
        else: Precio = {"Precio":driver.find_element(By.XPATH, '//*[@id="site-content"]/div/div[1]/div[3]/div/div[2]/div/div/div[1]/div/div/div/div/div/div/div[1]/div[1]/div[1]/div/span/div/span[2]').text}
    except: Precio = {"Precio": None}
    try: Nota = {"Nota":driver.find_element(By.XPATH, '//*[@id="site-content"]/div/div[1]/div[3]/div/div[2]/div/div/div[1]/div/div/div/div/div/div/div[1]/div[1]/div[2]/span/span[2]').text}
    except: Nota = {"Nota": None}
    try: Nnota = {"Nnota":driver.find_element(By.XPATH, '//*[@id="site-content"]/div/div[1]/div[3]/div/div[2]/div/div/div[1]/div/div/div/div/div/div/div[1]/div[1]/div[2]/span/span[3]/button/span').text}
    except: Nnota = {"Nota": None}
    try: Descripción = {"Descripción":driver.find_element(By.XPATH, '//*[@data-section-id="DESCRIPTION_DEFAULT"]/div/div/span/span').text}
    except: Descripción = {"Descripción":None}
    Titulo1 = {"Titulo1":driver.find_element(By.XPATH, '//*[@id="site-content"]/div/div[1]/div[1]/div[1]/div/div/div/div/section/div[1]/span/h1').text}
    Titulo2 = {"Titulo2":driver.find_element(By.XPATH, '//*[@id="site-content"]/div/div[1]/div[3]/div/div[1]/div/div[1]/div/div/section/div/div/div/div[1]/div/h2').text}
    Servicios = {"Servicios":driver.find_element(By.XPATH, '//*[@data-section-id="AMENITIES_DEFAULT"]/section/div[3]/div').text}
    Anfitrion = {"Anfitrion":driver.find_element(By.XPATH, '//*[@id="site-content"]/div/div[1]/div[6]/div/div/div/div[2]/div/section/div[1]/div[2]/h2').text}
    Registro = {"Registro":driver.find_element(By.XPATH, '//*[@id="site-content"]/div/div[1]/div[6]/div/div/div/div[2]/div/section/div[1]/div[2]/div/ol/li').text}
    try: Ubides = {"Ubides":driver.find_element(By.XPATH, '//*[@id="site-content"]/div/div[1]/div[5]/div/div/div/div[2]/div/section/div[4]/div/div/div/div[2]/span/span').text}
    except: Ubides = {"Ubides":None}
    Ubicacion = {"Ubicacion":driver.find_element(By.XPATH, '//*[@class="gm-style"]//a').get_attribute("href")}
    n_free_date = {"n_free_date": driver.find_element(By.XPATH, '//div[contains(text(),"Llegada")]/../div[contains(@data-testid, "change-dates")]').text}
    ScrapDate = {"ScrapDate": datetime.now().date().strftime("%d-%m-%Y")}
    dicts = [Link, Camas, Baños, Capacidad, Habitaciones, Precio, Nota, Nnota,
        Descripción, Titulo1, Titulo2, Servicios, Anfitrion, Registro, Ubides, Ubicacion, n_free_date, ScrapDate]
    #print(dicts)
    for dic in dicts:
        Comuna.update(dic)

    dfmin = pd.DataFrame([Comuna])
    df = pd.concat([df,dfmin])
    return df

In [100]:
df = pd.DataFrame()
def scrap_items(driver, df):
    initial_url = driver.current_url
    #print(type(initial_url))
    #print(initial_url)
    items = driver.find_elements(By.XPATH, '//div[contains(@style, "contents")]/.//div[@itemprop="itemListElement"]/div/div/div/a')
    print(len(items))
    links = []
    for item in items:
        #print(item.get_attribute("href"))
        links.append(item.get_attribute("href"))
    for link in links:
        try: df = scrap(driver, link, df)
        except: 
            print("POP UP ACTIVO")
            time.sleep(10)
            cerrar_button = driver.find_element(By.XPATH, "//button[@aria-label='Cerrar']")
            cerrar_button.click()
            time.sleep(3)
            df = scrap(driver, link, df)

    driver.get(initial_url)

    return df

df = scrap_items(driver, df)

0


In [101]:
def siguiente_pag(driver):
    button = driver.find_element(By.XPATH, '//*[@aria-label="Siguiente"]')
    print(button)
    button.click()

siguiente_pag(driver)

### CUANDO NO QUEDAN PAGS, AIRBNB EMPIEZA A RECOMENDAR OTRAS OFERTAS. AHÍ PARA CERRAR EL LOOP SE HACE CUANDO LOS LINKS SON = 0.

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@aria-label="Siguiente"]"}
  (Session info: chrome=109.0.5414.121)
Stacktrace:
Backtrace:
	(No symbol) [0x00766643]
	(No symbol) [0x006FBE21]
	(No symbol) [0x005FDA9D]
	(No symbol) [0x00631342]
	(No symbol) [0x0063147B]
	(No symbol) [0x00668DC2]
	(No symbol) [0x0064FDC4]
	(No symbol) [0x00666B09]
	(No symbol) [0x0064FB76]
	(No symbol) [0x006249C1]
	(No symbol) [0x00625E5D]
	GetHandleVerifier [0x009DA142+2497106]
	GetHandleVerifier [0x00A085D3+2686691]
	GetHandleVerifier [0x00A0BB9C+2700460]
	GetHandleVerifier [0x00813B10+635936]
	(No symbol) [0x00704A1F]
	(No symbol) [0x0070A418]
	(No symbol) [0x0070A505]
	(No symbol) [0x0071508B]
	BaseThreadInitThunk [0x75C000F9+25]
	RtlGetAppContainerNamedObjectPath [0x76F47BBE+286]
	RtlGetAppContainerNamedObjectPath [0x76F47B8E+238]


In [ ]:
df["Nnota"]

0           NaN
0    26 reseñas
0    12 reseñas
0    45 reseñas
0     3 reseñas
0    23 reseñas
0     3 reseñas
0     8 reseñas
0    11 reseñas
0    10 reseñas
0           NaN
0     4 reseñas
0     5 reseñas
0           NaN
0     3 reseñas
0           NaN
0     3 reseñas
0     3 reseñas
Name: Nnota, dtype: object

In [ ]:
unicos_columna(df, 2)

DATOS COLUMNA  Camas
5 camas    7
4 camas    6
6 camas    3
9 camas    1
3 camas    1
Name: Camas, dtype: int64


In [9]:
def pre_procesado(df):
    dfpre = df.copy(True)
    dfpre['Comuna'] = df['Comuna'].apply(lambda x: x.split(',')[0])
    dfpre["Camas"] = df["Camas"].apply(lambda x:x.replace(" ","").replace("camas","").replace("cama",""))
    dfpre["Baños"] = df["Baños"].apply(lambda x:x.replace(" ","").replace("baños","").replace("baño",""))
    dfpre["Tipo_Baños"] = df["Baños"].apply(lambda x: 'compartido' if 'compartido' in x else 'privado')
    dfpre["Capacidad"] = df["Capacidad"].apply(lambda x:x.replace(" huéspedes","").replace(" huésped",""))
    def remove_numbers(input_string):
        return input_string.translate(str.maketrans("", "", string.digits))
    dfpre["Habitaciones"] = df["Habitaciones"].apply(remove_numbers)
    dfpre["Habitaciones"] = dfpre["Habitaciones"].apply(lambda x:x.replace("habitaciones","Habitación"))
    dfpre["N_Habitaciones"] = df["Habitaciones"].apply(lambda x: '1' if 'Estudio' in x else x.replace(" habitaciones","").replace(" habitación",""))
    dfpre["Precio"] = df["Precio"].apply(lambda x:x.replace(" CLP","").replace("$","").replace(",",""))
    #dfpre["Precio"] = df["Precio"].str.extract(r'(\d+(?:,\d+)?)')
    dfpre["Nota"] = df["Nota"].apply(lambda x: x if x is None else x.replace(" ·", ""))
    dfpre["Nnota"]  = df["Nnota"].apply(lambda x: str(x).replace(" reseñas", "").replace(" reseña","") if not pd.isnull(x) else x)
    dfpre["Titulo2"] = df['Titulo2'].apply(lambda x: x.split(' - ')[0])
    dfpre["Servicios"] = df["Servicios"].apply(lambda x: x.split(' - '))
    dfpre["Anfitrion"] = df["Anfitrion"].apply(lambda x:x.replace("Anfitrión: ",""))
    print(df.shape)
    print(dfpre.shape)
    dfpre["Registro"] = df["Registro"].apply(lambda x:x.replace("Se registró en ","").replace(" de ","-"))
    def transform_string(s):
        months = {
        "enero": "01",
        "febrero": "02",
        "marzo": "03",
        "abril": "04",
        "mayo": "05",
        "junio": "06",
        "julio": "07",
        "agosto": "08",
        "septiembre": "09",
        "octubre": "10",
        "noviembre": "11",
        "diciembre": "12"
        }
        month, year = s.split("-")
        return f"{months[month]}-{year}"

    dfpre["Registro"] = dfpre["Registro"].apply(transform_string)
    dfpre[["lat", "lon"]] = df["Ubicacion"].str.extract(r"ll=(-\d+\.\d+),(-\d+\.\d+)")
    
    cols = ['Titulo1', 'Titulo2', 'Comuna',  'Capacidad',  'Camas', 'N_Habitaciones', 'Habitaciones', 'Baños',  'Tipo_Baños', 'Precio',  'lat', 'lon', 'Ubicacion', 'Ubides', 'Descripción', 'Servicios', 'Nota', 'Nnota', 'Anfitrion', 'Registro', 'Link']
    dfpre = dfpre[cols].reset_index()
    dfpre = dfpre.drop('index', axis=1)
    return dfpre

dfpre = pre_procesado(df)
dfpre.head()


NameError: name 'df' is not defined

In [ ]:
dfpre["Link"][0]

'https://www.airbnb.cl/rooms/692813718919764807?adults=1&children=0&infants=0&pets=0&check_in=2023-03-03&check_out=2023-03-08&previous_page_section_name=1000'

In [103]:
df = pd.DataFrame()
df = scrap(driver, "https://www.airbnb.cl/rooms/806383910552422838?adults=1&children=0&infants=0&pets=0&check_in=2023-02-13&check_out=2023-02-19&previous_page_section_name=1000", df)

{'Link': 'https://www.airbnb.cl/rooms/806383910552422838?adults=1&children=0&infants=0&pets=0&check_in=2023-02-13&check_out=2023-02-19&previous_page_section_name=1000'}


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="site-content"]/div/div[1]/div[3]/div/div[2]/div/div/div[1]/div/div/div/div/div/div/div[1]/div[1]/div[1]/div/span/div/span[2]"}
  (Session info: chrome=109.0.5414.121)
Stacktrace:
Backtrace:
	(No symbol) [0x00766643]
	(No symbol) [0x006FBE21]
	(No symbol) [0x005FDA9D]
	(No symbol) [0x00631342]
	(No symbol) [0x0063147B]
	(No symbol) [0x00668DC2]
	(No symbol) [0x0064FDC4]
	(No symbol) [0x00666B09]
	(No symbol) [0x0064FB76]
	(No symbol) [0x006249C1]
	(No symbol) [0x00625E5D]
	GetHandleVerifier [0x009DA142+2497106]
	GetHandleVerifier [0x00A085D3+2686691]
	GetHandleVerifier [0x00A0BB9C+2700460]
	GetHandleVerifier [0x00813B10+635936]
	(No symbol) [0x00704A1F]
	(No symbol) [0x0070A418]
	(No symbol) [0x0070A505]
	(No symbol) [0x0071508B]
	BaseThreadInitThunk [0x75C000F9+25]
	RtlGetAppContainerNamedObjectPath [0x76F47BBE+286]
	RtlGetAppContainerNamedObjectPath [0x76F47B8E+238]


In [8]:
tipo = pd.DataFrame({"YES":[None,"$100.000"]})
print(tipo["YES"].apply(lambda x: x if x is None else x.extract(r'(\d+(?:,\d+)?)')))

AttributeError: 'str' object has no attribute 'extract'